In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

import numpy as np
from sklearn.linear_model import LogisticRegression

- WordNet® is a large lexical database of English.
> https://wordnet.princeton.edu/

In [2]:
# Lemmatization
# turn words into base form
# cats > cat
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
# from http://www.lextek.com/manuals/onix/stopwords1.html

# rstrip: 오른쪽 공백('\n') 제거
# set: python 집합 자료형 지정, set은 중복을 허용하지 않고, 순서 없음
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [4]:
# load the reviews
# data courtesy of http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html

# xml file
# 파일 인코딩 ANSI로 변경
# 메모장으로 열고 인코딩 변경 후 다른 이름으로 저장

positive_reviews = BeautifulSoup(open('electronics/positive_ansi.review').read())
positive_reviews = positive_reviews.findAll('review_text')

negative_reviews = BeautifulSoup(open('electronics/negative_ansi.review').read())
negative_reviews = negative_reviews.findAll('review_text')

C:\Users\cwh\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
positive_reviews[0]

<review_text>
I purchased this unit due to frequent blackouts in my area and 2 power supplies going bad.  It will run my cable modem, router, PC, and LCD monitor for 5 minutes.  This is more than enough time to save work and shut down.   Equally important, I know that my electronics are receiving clean power.

I feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply.

As always, Amazon had it to me in &lt;2 business days
</review_text>

In [6]:
negative_reviews[0]

<review_text>
cons
tips extremely easy on carpet and if you have a lot of cds stacked at the top

poorly designed, it is a vertical cd rack that doesnt have individual slots for cds, so if you want a cd from the bottom of a stack you have basically pull the whole stack to get to it

putting it together was a pain, the one i bought i had to break a piece of metal just to fit it in its guide holes.

again..poorly designed... doesnt even fit cds that well, there are gaps, and the cd casses are loose fitting

pros
..........
i guess it can hold a lot of cds....
</review_text>

In [7]:
# there are more positive reviews than negative reviews
# so let's take a random sample so we have balanced classes

# 긍정, 부정 리뷰 개수 일치 시키기
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]
print(len(positive_reviews))

1000


In [8]:
# first let's just try to tokenize the text using nltk's tokenizer
# let's take the first review for example:

# 단순 공백 기반 토큰화
t = positive_reviews[0]
nltk.tokenize.word_tokenize(t.text)

['The',
 'Sennheiser',
 'HD600s',
 'have',
 'only',
 'one',
 'thing',
 'wrong',
 'with',
 'them',
 ':',
 'they',
 "'ll",
 'change',
 'you',
 'from',
 'someone',
 'who',
 'enjoys',
 'good',
 'audio',
 '...',
 'into',
 'an',
 'audiophile',
 '!',
 'I',
 "'ve",
 'been',
 'known',
 'as',
 'a',
 'videophile',
 'for',
 'many',
 'years',
 'over',
 'on',
 'one',
 'of',
 'the',
 'top',
 'AV',
 'forums',
 ',',
 'but',
 'I',
 "'ve",
 'always',
 'been',
 'satisfied',
 'with',
 'merely',
 '``',
 'decent',
 "''",
 'audio',
 '.',
 'Until',
 'now',
 '.',
 'I',
 "'d",
 'previously',
 'ripped',
 'all',
 'my',
 'CDs',
 'into',
 'the',
 '(',
 'previously',
 'quite',
 'impressive',
 ')',
 'Windows',
 'Media',
 'format',
 'at',
 '192K',
 'bps',
 ',',
 'but',
 'that',
 "'s",
 'now',
 'useless',
 '.',
 'With',
 'the',
 'phenomenal',
 'ability',
 'of',
 'the',
 'Sennheiser',
 'HD600s',
 'to',
 'reveal',
 'all',
 'the',
 'subtle',
 'nuances',
 'of',
 'your',
 'music',
 ',',
 'listening',
 'to',
 'compressed',
 '

In [9]:
# notice how it doesn't downcase, so It != it
# not only that, but do we really want to include the word "it" anyway?
# you can imagine it wouldn't be any more common in a positive review than a negative review
# so it might only add noise to our model.
# so let's create a function that does all this pre-processing for us

# 전처리 함수

def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

In [10]:
# create a word-to-index map so that we can create our word-frequency vectors later
# let's also save the tokenized versions so we don't have to tokenize again later

# 전체 token 
word_index_map = {}
current_index = 0

positive_tokenized = []
negative_tokenized = []

In [11]:
# nltk.download()

In [12]:
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [13]:
# 첫 등장 index
word_index_map

{'tightening': 10998,
 'shure': 1511,
 'chip': 4038,
 'openness': 7179,
 'drove': 5606,
 'bourne': 5081,
 'hot': 232,
 'fruitless': 9198,
 'flipped': 4952,
 'brighter': 1628,
 'kitchen': 799,
 'smoking': 11068,
 'aspect': 3395,
 'corupt': 7964,
 'lab': 2620,
 'composite': 3775,
 'table': 4107,
 'you': 5,
 'clench': 10857,
 'array': 9389,
 'motorala': 7066,
 'friendly': 459,
 'chronically': 2145,
 'vey': 6879,
 'economical': 6303,
 'untrue': 8187,
 'lucky': 3962,
 'chane': 3794,
 '1900': 7017,
 "n't": 271,
 'elevation': 3468,
 'print': 861,
 'programed': 1708,
 'introduced': 6262,
 'slim': 1003,
 'cooler': 4545,
 'tomita': 461,
 'pita': 7129,
 'design/quality': 5729,
 'race': 3295,
 'clamp': 10697,
 'homesick': 6104,
 'allover': 3714,
 'lense': 7093,
 'unwooded': 10307,
 'dashmount': 7154,
 'a**': 6255,
 'funeral': 1615,
 'deny': 8037,
 'recieved': 2784,
 'asa': 6635,
 'uploaded': 9062,
 'enterntainment': 2861,
 'msc': 10839,
 'edt': 9028,
 'offset': 5671,
 'form-factor': 5500,
 'edison

In [14]:
print(len(positive_tokenized))
print(positive_tokenized[0])

1000
['sennheiser', 'hd600s', 'wrong', "'ll", 'change', 'you', 'enjoys', 'audio', '...', 'audiophile', "'ve", 'videophile', 'top', 'forum', "'ve", 'satisfied', 'merely', 'decent', 'audio', 'previously', 'ripped', 'cd', 'previously', 'impressive', 'window', 'medium', 'format', '192k', 'bps', 'useless', 'phenomenal', 'ability', 'sennheiser', 'hd600s', 'reveal', 'subtle', 'nuance', 'music', 'listening', 'compressed', 'disc', 'listening', 'music', 'thick', 'layer', 'vaseline', '...', 'time', 'rip', 'lossless', 'format', 'recommend', 'headphone', 'highly', 'simply', 'extraordinary', 'taste', 'accustomed', 'this', 'level', 'quality', 'life', 'lot', 'expensive', 'you', 'try', 'update', 'match', 'level', 'sonic', 'differentiation', 'this', 'bloody', 'miracle', 'ear', 'finally', 'wow']


- 전체 토큰 개수 (len word_index_map) + 1 의 크기를 가진 0으로 이루어진 배열 생성
- 토큰이 등장 시 해당 인덱스의 0 을 1로 변경
- 예: (0, 1, 1, 0, 0, ...) - 긍정 토큰 등장 위치 / (1, 0, 0, 1, 1, ...) - 부정 토큰 등장 위치
- normalization 진행
- 마지막에 label (1 positive), (0 negative) 추가 
- 예: (0,1, 0,1, ... , 1) - 긍정 토큰에 대한 벡터

In [15]:
# now let's create our input matrices
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # last element is for the label
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum() # normalize it before setting label
    x[-1] = label # last element to label
    return x

- N(긍정토큰개수+부정토큰개수) x (전체토큰개수+label) 크기의 배열 생성

In [16]:
N = len(positive_tokenized) + len(negative_tokenized)
# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later

data = np.zeros((N, len(word_index_map) + 1))
i = 0

- i 번째열에 해당 토큰의 벡터 입력, 마지막에는 label 1(긍정), 0(부정)

In [17]:
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

In [18]:
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [19]:
data.shape

(2000, 11087)

In [20]:
# shuffle the data and create train/test splits
# try it multiple times!
np.random.shuffle(data)

X = data[:,:-1]
Y = data[:,-1]

# last 100 rows will be test
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

model = LogisticRegression()
model.fit(Xtrain, Ytrain)
# score : accuracy
print ("Classification rate:", model.score(Xtest, Ytest))

Classification rate: 0.74


In [21]:
print(len(model.coef_[0]))
print(model.coef_[0][1350])

11086
0.0277897279989


In [22]:
# let's look at the weights for each word
# try it with different threshold values!
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print (word, weight)

you 1.00723593281
n't -2.05927924432
item -0.9412947787
warranty -0.539083645795
memory 0.918705104163
highly 1.01878791991
perfect 0.932518584341
look 0.507636080956
quality 1.47482218451
software -0.521852277378
happy 0.562334518808
time -0.598524294892
value 0.565074205184
then -1.08211606214
returning -0.502076779899
hour -0.555298038
try -0.652303780386
poor -0.768485883586
fast 0.874540001256
comfortable 0.619045442121
buy -0.876390820465
using 0.687123846856
customer -0.70505658046
month -0.668462193474
've 0.723438936124
picture 0.571266970643
laptop 0.506524282577
pretty 0.658956858729
support -0.881874041879
excellent 1.39593231081
little 0.981372889034
unit -0.582974099882
tried -0.819728765417
space 0.586582626606
bit 0.617250946931
love 1.24922201042
refund -0.571711366548
returned -0.814800123209
radio -0.507453080066
lot 0.728392931595
sound 1.02512822541
terrible -0.513362389709
junk -0.518726190441
speaker 0.9249128526
week -0.693881178847
ha 0.641085137076
recommend 0